In [ ]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia numexpr rizaio

Python(60073) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [84]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools.riza.command import ExecPython
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
import traceback
from langchain_community.utilities import GoogleSerperAPIWrapper

In [85]:
def exec_python_code(code: str) -> str:
    """Executes Python code and returns the output or error message."""
    local_vars = {}
    try:
        exec(code, {}, local_vars)  # Execute code in an isolated scope
        return str(local_vars)  # Return local variables as the result
    except Exception as e:
        return str(traceback.format_exc())  # Return error traceback if execution fails

# Define the tool
exec_python_tool = Tool(
    name="Exec Python",
    func=exec_python_code,
    description="Executes arbitrary Python code and returns the output."
)

# Example usage
result = exec_python_tool.run('''x = 5\ny = x * 2\nresult = y''')
print(result)  # Output should include {'x': 5, 'y': 10, 'result': 10}

{'x': 5, 'y': 10, 'result': 10}


In [86]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPER_DEV_API_KEY")


llm_ollama= ChatOpenAI(
    model_name="llama3.2:3b",
    base_url="http://localhost:11434/v1",
)

llm_openai = ChatOpenAI(
    model_name="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
)

llm_groq = ChatOpenAI(
    model_name="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.getenv("GROQ_API_KEY"),
)

llm  = llm_openai

# Initialize the DuckDuckGo search tool
ddg_search = DuckDuckGoSearchRun()

ddg_search_tool = Tool(
    name="DuckDuckGo Search",
    func=ddg_search.run,
    description="A tool for performing web searches using DuckDuckGo."
)

serper_search = GoogleSerperAPIWrapper(serper_api_key= os.getenv("SERPER_DEV_API_KEY"))
serper_search_tool = Tool(
    name="Serper Search",
    func=serper_search.run,
    description="A tool for performing web searches using Serper.dev."
)

# Load standard tools
standard_tools = load_tools(["llm-math","wikipedia","terminal"], llm=llm,allow_dangerous_tools=True)

# Combine standard tools with the custom DuckDuckGo search tool
#tools = standard_tools + [ddg_search_tool, exec_python_tool]
tools = standard_tools + [serper_search_tool, exec_python_tool]

In [87]:
# What is a tool like this
for i in range(len(tools)):
    print(f"{tools[i].name}: {tools[i].description}")

Calculator: Useful for when you need to answer questions about math.
wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
terminal: Run shell commands on this MacOS machine.
Serper Search: A tool for performing web searches using Serper.dev.
Exec Python: Executes arbitrary Python code and returns the output.


In [88]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(tools, 
                         llm, 
                         agent="zero-shot-react-description", 
                         handle_parsing_errors=True,
                        max_iterations=15,  # Set to an appropriate number
                        memory = memory,
                        verbose=True)

In [89]:
# Correct input format as a dictionary
input_data = "Hi, how are you today?"

# Run the agent with the properly formatted input
response = agent.run(input_data)

# Output the response
print(response)



> Entering new AgentExecutor chain...
I don't have feelings, but I'm here and ready to assist you! How can I help you today? 
Final Answer: I'm here and ready to assist you! How can I help you today?

> Finished chain.
I'm here and ready to assist you! How can I help you today?


In [68]:
agent.run("Who is the current United States President in 2025? What is his current age raised divided by 2?")



> Entering new AgentExecutor chain...
To answer the first part of the question, I need to determine who the current United States President is in 2025. Since I have data only up to October 2023, I cannot provide real-time updates or future predictions. However, I can perform a web search to find the current United States President in 2025.

For the second part, I will need to know the birth year of the President to calculate his age in 2025 and then divide that by 2. 

Action: Serper Search  
Action Input: "current United States President in 2025"  
Observation: United States: Country in North America. United States Government: Federation, Presidential system, Federal republic, and more. United States Capital: Washington, D.C.. United States Fertility rate: 1.66 births per woman (2022). United States GDP growth rate: 2.5% annual change (2023). United States Gross domestic product: 27.36 trillion USD (2023). United States GDP per capita: 81,695.19 USD (2023). United States Gross natio

'The current United States President in 2025 is Donald Trump, and his age divided by 2 is 39.5.'

In [73]:
agent.run("Create a blog article on Swami Vivekananda based on wikipedia entries about him. Calculate the number of words in this article which you created.")



> Entering new AgentExecutor chain...
To create a blog article on Swami Vivekananda, I'll first gather information from Wikipedia about him. After obtaining the relevant content, I'll then calculate the number of words in the article.

Action: wikipedia  
Action Input: "Swami Vivekananda"  
Observation: Page: Swami Vivekananda
Summary: Swami Vivekananda (12 January 1863 – 4 July 1902), born Narendranath Datta was an Indian Hindu monk, philosopher, author, religious teacher, and the chief disciple of the Indian mystic Ramakrishna. He was a key figure in the introduction of Vedanta and Yoga to the Western world. He is credited with raising interfaith awareness and bringing Hinduism to the status of a major world religion in the late nineteenth century.
Born into an aristocratic Bengali Kayastha family in Calcutta, Vivekananda was inclined from a young age towards religion and spirituality. At the age of 18 he met Ramakrishna, later becoming a devoted follower and sannyasin (renunciate)

KeyboardInterrupt: 

In [74]:
agent.run("Get me a summary of a wikipedia article on deepmind.")



> Entering new AgentExecutor chain...
I need to look up the Wikipedia article on DeepMind to provide a summary. 
Action: wikipedia
Action Input: "DeepMind"

/Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Google DeepMind
Summary: DeepMind Technologies Limited, trading as Google DeepMind or simply DeepMind, is a British-American artificial intelligence research laboratory which serves as a subsidiary of Alphabet Inc. Founded in the UK in 2010, it was acquired by Google in 2014 and merged with Google AI's Google Brain division to become Google DeepMind in April 2023. The company is headquartered in London, with research centres in the United States, Canada, France, and Germany.
DeepMind introduced neural Turing machines (neural networks that can access external memory like a conventional Turing machine), resulting in a computer that loosely resembles short-term memory in the human brain.
DeepMind has created neural network models to play video games and board games. It made headlines in 2016 after its AlphaGo program beat a human professional Go player Lee Sedol, a world champion, in a five-game match, which was the subject of a documentary film. A more general program

'DeepMind Technologies Limited, also known as Google DeepMind, is a British-American artificial intelligence research lab and a subsidiary of Alphabet Inc. Founded in 2010 and acquired by Google in 2014, it focuses on various AI advancements, including neural networks and reinforcement learning. Notable achievements include the development of AlphaGo, which defeated world champion Go player Lee Sedol in 2016, and AlphaFold, which made significant strides in protein folding. The company has a global presence with research centers in multiple countries.'

In [75]:
agent.run("What files are in my current directory?")



> Entering new AgentExecutor chain...
I need to check the current directory to see what files are present.  
Action: terminal  
Action Input: ls  Executing command:
 ls

Observation: Week_1
Week_2
Week_3
Week_4
chroma_db
genai_env

Thought:

/Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(
Python(60361) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


I now know the final answer.  
Final Answer: The current directory contains the following files and folders: Week_1, Week_2, Week_3, Week_4, chroma_db, genai_env.

> Finished chain.


'The current directory contains the following files and folders: Week_1, Week_2, Week_3, Week_4, chroma_db, genai_env.'

In [90]:
agent.run("How much system ram is available on my mac?")



> Entering new AgentExecutor chain...
I need to check the available system RAM on the Mac. I can do this using a terminal command that retrieves system memory information. 

Action: terminal  
Action Input: `vm_stat`  Executing command:
 `vm_stat`

Observation: /bin/sh: Mach: command not found

Thought:

/Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(
Python(60573) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


It seems that the command I tried to execute did not work. I will try a different command to check the available system RAM on the Mac.

Action: terminal  
Action Input: `system_profiler SPHardwareDataType | grep Memory`  Executing command:
 `system_profiler SPHardwareDataType | grep Memory`


/Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(
Python(60578) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



Observation: /bin/sh: Memory:: command not found

Thought:It seems that the terminal commands I have tried to execute are not working properly. I will try a different approach to retrieve the available system RAM.

Action: terminal  
Action Input: `sysctl -a | grep mem`  Executing command:
 `sysctl -a | grep mem`

Observation: /bin/sh: kern.ipc.mb_memory_pressure_percentage:: command not found

Thought:

/Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(
Python(60589) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


It appears that I am unable to execute terminal commands directly. Instead, I can use alternative methods to find the available system RAM. I will try searching online for how to check the available RAM on a Mac.

Action: Serper Search  
Action Input: "how to check available RAM on Mac"  
Observation: In the Activity Monitor app on your Mac, click Memory to see the following in the bottom of the window: · To display more columns, choose View > Columns, then ... In the Activity Monitor app on your Mac, click Memory. The Memory Pressure graph lets you know if your computer is using memory efficiently. Press Ctrl + Shift + Esc to launch Task Manager. Or, right-click the Taskbar and select Task Manager. · Select the Performance tab to see current ... ... Mac” in the Apple menu, home in on what version of macOS you're running, and locate your available gigabytes of RAM. You'll also learn where ... Want to check your Mac's RAM? Here's how to do it in simple steps: Step 1. On your Mac device,

'To check the available system RAM on your Mac, open the Activity Monitor (found in Applications > Utilities), then click on the "Memory" tab. You can view the memory pressure and the amount of RAM being used at the bottom of the window.'

In [91]:
agent.run("What is the current price of nifty50?")



> Entering new AgentExecutor chain...
To find the current price of Nifty 50, I should perform a web search since this is financial data that can change frequently. 

Action: Serper Search  
Action Input: "current price of Nifty 50"  
Observation: 23,045.25 -26.55 (0.12%)
Thought:I have obtained the current price of Nifty 50 from the web search. 

Final Answer: The current price of Nifty 50 is 23,045.25, down by 26.55 (0.12%).

> Finished chain.


'The current price of Nifty 50 is 23,045.25, down by 26.55 (0.12%).'

In [93]:
agent.run("I can run 10 KM/Hour and swim at 3 KM per hour. I have to go through a road of 5 Km followed by 6 KM of river. How much time will it take?")



> Entering new AgentExecutor chain...
To calculate the total time taken for the journey, I need to find the time taken to cover each segment of the trip: the road and the river. 

1. For the road (5 km) at a speed of 10 km/h, I can use the formula:
   \[
   \text{Time} = \frac{\text{Distance}}{\text{Speed}} = \frac{5 \text{ km}}{10 \text{ km/h}}
   \]

2. For the river (6 km) at a speed of 3 km/h, I can use the same formula:
   \[
   \text{Time} = \frac{\text{Distance}}{\text{Speed}} = \frac{6 \text{ km}}{3 \text{ km/h}}
   \]

I will calculate both times and then sum them to find the total time.

Action: Exec Python code to calculate the total time.
Action Input: 
```python
road_distance = 5  # in km
road_speed = 10    # in km/h
river_distance = 6  # in km
river_speed = 3     # in km/h

road_time = road_distance / road_speed
river_time = river_distance / river_speed

total_time = road_time + river_time
total_time
``` 
Observation: Exec Python code to calculate the total time. is not

'2.5 hours'

In [94]:
prompt_template = agent.agent.llm_chain.prompt
print(prompt_template)

input_variables=['agent_scratchpad', 'input'] input_types={} partial_variables={} template='Answer the following questions as best you can. You have access to the following tools:\n\nCalculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\nterminal - Run shell commands on this MacOS machine.\nSerper Search(query: str, **kwargs: Any) -> str - A tool for performing web searches using Serper.dev.\nExec Python(code: str) -> str - Executes arbitrary Python code and returns the output.\n\nUse the following format:\n\nQuestion: t

input_variables=['agent_scratchpad', 'input'] 
input_types={} 
partial_variables={} 
template="Answer the following questions as best you can."
"You have access to the following tools:\n\n"
"Calculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler],"
"langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,"
"tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.\n"
"wikipedia - A wrapper around Wikipedia."
"Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects."
"Input should be a search query.\nterminal - Run shell commands on this MacOS machine.\n"
"DuckDuckGo Search(tool_input: 'Union[str, dict[str, Any]]', verbose: 'Optional[bool]' = None,"
"start_color: 'Optional[str]' = 'green', color: 'Optional[str]' = 'green', callbacks:"
"'Callbacks' = None, *, tags: 'Optional[list[str]]' = None, metadata: 'Optional[dict[str, Any]]' = None,"
"run_name: 'Optional[str]' = None, run_id: 'Optional[uuid.UUID]' = None, config:"
"'Optional[RunnableConfig]' = None, tool_call_id: 'Optional[str]' = None, **kwargs: 'Any')"
"-> 'Any' - A tool for performing web searches using DuckDuckGo.\n\n"
"Use the following format:\n\n"
"Question: the input question you must answer\n"
"Thought: you should always think about what to do\n"
"Action: the action to take, should be one of [Calculator, wikipedia, terminal, DuckDuckGo Search]\n"
"Action Input: the input to the action\n"
"Observation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\n"
"Thought: I now know the final answer\n"
"Final Answer: the final answer to the original input question\n\n"
"Begin!\n\n"
"Question: {input}\nThought:{agent_scratchpad}"

In [95]:
agent.run("I am in south bangalore near uttarahalli main road. I have got a baby who is having cough. Which doctor can I consult with?")



> Entering new AgentExecutor chain...
I need to find doctors or clinics in South Bangalore, specifically near Uttarahalli Main Road, who specialize in treating children or have experience with pediatric cough. I will perform a web search to find this information.

Action: Serper Search  
Action Input: "pediatrician near Uttarahalli Main Road South Bangalore for cough"  
Observation: Best Pediatricians In Uttarahalli, Bangalore ; Dr. Eash Hoskote Pediatrician in Bangalore Dr. Eash Hoskote · 28 years experience ; Dr. Lingaraj Mulage Pediatrician ... Missing: cough | Show results with:cough. Best Paediatricians in Uttarahalli, Bangalore - Book Appointment Online · Rainbow Childrens Hospital & Birthright By Rainbow · Apollo Cradle & Childrens Hospital. Best Pediatricians in · Uttarahalli Bangalore · Best Pediatrician near Uttarahalli, Bengaluru · Dr. Nilesh Rao · Dr. Arjun S K · Dr. Anand Alladi · Dr. Ravneet ... Missing: South | Show results with:South. P S Ragavan. Paediatrician. 28 Ye

"You can consult **Dr. Eash Hoskote** or visit **Rainbow Children's Hospital** in Uttarahalli for your baby's cough. Alternatively, **Dr. Arjun S K** and **Dr. Nilesh Rao** are also good options in the area."

In [96]:
agent.run("What is the average rent of 2 bedroom appartment in uttarahalli, bangalore area?")



> Entering new AgentExecutor chain...
I need to find the current average rent for a 2-bedroom apartment in the Uttarahalli area of Bangalore. The best approach is to perform a web search to gather recent data on rental prices in that location. 

Action: Serper Search  
Action Input: "average rent 2 bedroom apartment Uttarahalli Bangalore 2023"  
Observation: Rent 2 BHK Semi furnished Flat in Uttarahalli for Rs 38000 · Rent 2 BHK Semi furnished Flat in Uttarahalli for Rs 23000 · Rent 2 BHK Semi ... You can avail the 2 BHK flat for rent in the well-known location of Uttarahalli. The 2 BHK apartment is available at a reasonable rent of 35,000. Some premium ... Missing: average 2023. 1050 Sqft 2 BHK Flat available for rent in Uttarahalli Hobli, Bangalore by KRISHNA K. This 1050 Sqft 2 BHK 2toilet Flat at 15.0k with all amenities only on ... ₹30,000, 2 bhk Residential Apartment for rent in Subramanyapura - Entrance. ₹ 30,000 /month · Dewflower, ; Srivari Forest Breeze Bedroom. ₹ 38,000 ..

'The average rent of a 2-bedroom apartment in the Uttarahalli area of Bangalore is approximately ₹31,250 per month.'

In [97]:
agent.run("What is the average AQI of New Delhi over the last 5 days?")



> Entering new AgentExecutor chain...
To find the average Air Quality Index (AQI) of New Delhi over the last 5 days, I need to gather the daily AQI data for that period. I will perform a web search to find recent AQI statistics for New Delhi.

Action: Serper Search  
Action Input: "New Delhi AQI last 5 days"  
Observation: The AQI levels in New Delhi have been fluctuating throughout the last 24 hours. The levels have increased by highest 255 at 11:13 PM, Feb 11, 2025, lowest 95 at ... Delhi Air Quality Index (AQI) is now Hazardous. Get real-time, historical and forecast PM2.5 and weather data. Read the air pollution in Delhi, India with ... ... Delhi Real-time Air Quality Index (AQI). 483. Hazardous. Updated on Tuesday 23:00. Primary pollutant: pm10. current, past 2 days, min, max. PM2.5 AQI. 227 ... The AQI levels in Delhi have been fluctuating throughout the last 24 hours. The levels have increased by highest 195 at 12:13 AM, lowest 95 at 6:13 PM. Air Quality Index on Feb 11, 2025 

'The average AQI of New Delhi over the last 5 days is 186.2.'

In [43]:
# Correct input format as a dictionary
input_data = {"input": "What is the average AQI in New Delhi?"}

# Run the agent with the properly formatted input
response = agent.run(input_data)

# Output the response
print(response)



> Entering new AgentExecutor chain...
Good day, everyone! Welcome to your daily dose of weather sarcasm, where we take the temperature and turn it into a roast.

So, let’s dive into our current weather conditions. If you haven’t stepped outside yet, congratulations! You’re in for a real treat. Right now, the temperature is hovering around a balmy 75°F, which is just perfect for those of you who enjoy sweating profusely while standing still. Who needs a sauna, right?

And let’s talk about those clouds. Oh, they’re just hanging out, looking all fluffy and innocent, probably plotting to unleash a torrential downpour at any moment. Because who doesn’t love a surprise shower when you’re dressed to impress?

Winds? Oh, they’re out there too, blowing at a solid 10 mph. That’s just enough to mess up your hair but not quite enough to provide any real relief from the oppressive heat. How thoughtful of nature!

And don’t even get me started on the humidity. It's like Mother Nature decided to co

KeyboardInterrupt: 

In [98]:
from langchain.prompts import PromptTemplate

# Define the template with placeholders for dynamic content
template = """
Answer the following questions with your best sarcastic wit. Generate the final answer once you have the final answer using the tools. You have access to the following tools:

{tool_descriptions}

Use the following format:

Question: {input}
Thought: {agent_scratchpad}
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question, delivered with a touch of sarcasm

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

# Create a PromptTemplate instance
prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad", "tool_descriptions", "tool_names"],
    template=template,
)

# Assuming 'tools' is a list of Tool instances
tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
tool_names = ", ".join([tool.name for tool in tools])



In [99]:
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(input_key="input")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    agent_kwargs={
        'prefix': prompt.template,
        'input_variables': ["input", "agent_scratchpad", "tool_descriptions", "tool_names"],
    },
    memory=memory,
    max_iterations=20,
    verbose=True
)


In [100]:

response = agent.run({
    "input": "What's average rent in bangalore for 3bhk house?",
    "agent_scratchpad": "",
    "tool_descriptions": tool_descriptions,
    "tool_names": tool_names
})
print(response)



> Entering new AgentExecutor chain...
Since I’m clearly a real estate expert and not just a virtual assistant, let’s dive into the glamorous world of Bangalore rents, shall we? 

Action: Serper Search  
Action Input: "average rent for 3bhk house in Bangalore 2023"  
Observation: ... PROPERTY GROUPS. 3 BHK Flat for rent in Sarjapur, Bangalore. S S BRINDAVANAM. 3 BHK Flat for rent in Sarjapur, Bangalore. ₹42,000 / month. + Deposit ₹150,000. 3 bhk rent starts from rs 20k to 40k in middle and upper class area,if you want to buy the same class area can cost you between 1cr to 2.5 cr . Find 3 BHK flat for rent in JP Nagar Phase 5 Bangalore. 32,000 is the monthly rent of this 3 BHK flat. This 3 BHK rental flat comes with no brokerage fees. Missing: 2023 | Show results with:2023. Seeing the rents here makes me feel bad that we rented out our 3BHK for 32k near Kundanahalli. This is Semi furnihsed flat with the budget of 35000, Deposit 210000. More About This Property This is a well-designed 3